The trained LSTM model and the fitted scaler are being saved, then reload them for forecasting later.


1️ Train + Save

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib
import os

# Example: Fit scaler globally on all features
features_cols = ["market_value","total_injuries","sentiment_mean","avg_cards_per_match"]
scaler = MinMaxScaler()
scaler.fit(df[features_cols].fillna(0).values)

# Transform the data for model training
X, y = prepare_multivariate_sequences(df, scaler, features_cols, n_steps=3, n_future=1)

# Define LSTM model
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Create folder for saving
os.makedirs("model_artifacts", exist_ok=True)

# Save model + scaler
model.save("model_artifacts/lstm_model.h5")
joblib.dump(scaler, "model_artifacts/scaler.pkl")
print("✅ Model & scaler saved")


2️ Reload Later

In [ ]:
from tensorflow.keras.models import load_model
import joblib

# Load back model + scaler
model = load_model("model_artifacts/lstm_model.h5")
scaler = joblib.load("model_artifacts/scaler.pkl")
print("✅ Model & scaler loaded")

3️ Predict With Consistent Scale

In [ ]:
# Prepare new player’s data with same columns
player_df = get_player_data(transfermarkt_id)

features = player_df[features_cols].fillna(0).values
scaled_features = scaler.transform(features)

X_new, _ = make_multivariate_multistep(scaled_features, n_steps=3, n_future=1)

pred_scaled = model.predict(X_new)

# Inverse transform predictions (market_value)
dummy = np.zeros((pred_scaled.shape[0], len(features_cols)))
dummy[:,0] = pred_scaled[:,0]  # only first column = market_value
pred_original = scaler.inverse_transform(dummy)[:,0]

print("Predicted market values:", pred_original)

Same scaler is used everywhere.
Model file (`.h5`) and scaler file (`.pkl`) can be versioned together.
Easy to deploy in any Python environment.